# livedoor-newsデータを用いた5クラスの判別問題を 日本語BERTで90%超の精度を達成

---



# Fine tuning for livedoor-news-5class-classification by Japanese-BERT with accuracy of over 90%

---



In [ ]:
!nvidia-smi

Thu May 12 00:12:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. ライブラリをインポートします




## 1. Import library

In [ ]:
!pip install ktrain

In [ ]:
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

In [ ]:
import os
import h5py
import time
import glob
import json
import csv
import datetime
import random
import string
import pickle
import imageio
import cv2
import re
import sys
from tqdm import tqdm_notebook
from PIL import Image
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
#import tensorflow_text as text 
#from tensorflow_docs.vis import embed
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import ktrain
from ktrain import text
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm

from IPython.display import Image
%matplotlib inline

tf.random.set_seed(200) 

tf.__version__

'2.8.0'

## 2. livedoor-news dataを取り込みます


## 2. Load livedoor-news-data


In [ ]:
train_df = pd.read_csv("train5.tsv", delimiter='\t', header=None, names=['label', 'title'])
train_df=train_df
train_df

,label,title
0,0,"月額1,050円でSASディスクを搭載！クラウドプラン 「エクスライト（EX-LITE）」の..."
1,0,例のあのメニューも！ニコニコ超会議のフードコートメニュー14種類紹介（前半）
2,1,山田孝之と小西真奈美の映画に撮影していない“あるモノ”が映り込んでいた
3,0,ガガを聴きながら毎朝歯磨き！ レディー・ガガ シンギングトゥースブラシ独占先行販売
4,2,斎藤佑樹はプロで通用するのか？
...,...,...
3498,1,美女たちがドレスの競演をした東京国際映画祭！スケスケのシースルー衣装を着たセクシー女優も
3499,2,プロアマ問題に怒り心頭のノムさん、都知事選には「俺でもいけそう」
3500,3,【オトナ女子のリアルな悩み】バツイチでは恋はできませんか？
3501,2,丸山桂里奈、なでしこジャパンのゴールに笑顔なく「すごく複雑な気持ち」


In [ ]:
test_df = pd.read_csv("dev5.tsv", delimiter='\t', header=None, names=['label', 'title'])
test_df

,label,title
0,4,ドコモのｓｐモードでまたトラブル発生！ 今度はつながらない【話題】
1,2,ダルビッシュ、今季だけで3つの魔球体得へ
2,1,本格的に仕事復帰、マリエが今一番会いたい人は“スギちゃん”
3,0,価格が安価で矯正を気づかれない！綺麗な歯並びになれる、歯列矯正の最前線
4,2,先輩・桑田、松井秀喜にズバリ「引退とかは考えなかった？」
...,...,...
871,2,三浦皇成、ほしのあきとの交際認める
872,3,【オトナ女子コミック部】すべての女は“ブス”である。女の毒素が臭気を放つコミックエッセイ『だ...
873,4,気をつけて！ スマートフォン、海外パケット定額プランのつもりが約100万円！のナゾ【話題】
874,4,KDDIがiPhone向けアプリ「auお客さまサポート」 をリリース【話題】


## 3. 訓練データと検証データを作成します


## 3. Prepare training and test data


In [ ]:
X_train = train_df['title'].tolist()
y_train = train_df['label'].tolist()

In [ ]:
X_test = test_df["title"].tolist()
y_test = test_df['label'].tolist()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=100)

In [ ]:
print(type(x_train))
print(len(x_train))
print(type(y_train))
print(len(y_train))

<class 'list'>
3152
<class 'list'>
3152


In [ ]:
print(type(X_test))
print(len(X_test))
print(type(y_test))
print(len(y_test))

<class 'list'>
876
<class 'list'>
876


In [ ]:
x_train[0:3]

['『機動戦士ガンダムUC』最新作の主題歌に、新人が異例の大抜擢',
 'Time-Lapse（微速度）撮影ができるカメラ！ COMPTEX TAIPEIで見つけたオモシロガジェット【デジ通】',
 'タグ付けから友達管理まで\u3000Facebook活用のスゴ技・裏技テクニック集【知っ得！虎の巻】']

In [ ]:
y_train[0:3]

[1, 0, 0]

##  4. BERTモデルを構築します

##  4. Build BERT model

In [ ]:
maxlen = 100
lr = 2e-5
epochs = 5
batch = 32
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
classes = list(set(y_train))

In [ ]:
classes

[0, 1, 2, 3, 4]

データの前処理を実行します

preproces textdata

In [ ]:
t = text.Transformer(MODEL_NAME, maxlen=maxlen, class_names=classes)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_train(x_val, y_val)

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

preprocessing train...
language: ja


Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing train...
language: ja


Is Multi-Label? False


モデルを構築します

Build the model

In [ ]:
model = t.get_classifier()

Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

## 5. モデルを訓練します

## 5.Finetune the model

In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch)

t1=time.time()

learner.fit_onecycle(lr, epochs)

t2 = time.time()
print(round(t2-t1, 5), 'Seconds to predict')



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
99/99 [==============================] - 46s 264ms/step - loss: 1.4082 - accuracy: 0.4737 - val_loss: 0.8167 - val_accuracy: 0.8034
Epoch 2/5
99/99 [==============================] - 24s 235ms/step - loss: 0.4776 - accuracy: 0.8706 - val_loss: 0.2231 - val_accuracy: 0.9345
Epoch 3/5
99/99 [==============================] - 24s 236ms/step - loss: 0.1718 - accuracy: 0.9511 - val_loss: 0.2531 - val_accuracy: 0.9174
Epoch 4/5
99/99 [==============================] - 24s 235ms/step - loss: 0.0593 - accuracy: 0.9886 - val_loss: 0.1727 - val_accuracy: 0.9544
Epoch 5/5
99/99 [==============================] - 24s 235ms/step - loss: 0.0308 - accuracy: 0.9956 - val_loss: 0.1569 - val_accuracy: 0.9544
161.01792 Seconds to predict


## 6. モデルの精度を検証します

## 6. Evaluate the model

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
y_pred = predictor.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9419    0.9310    0.9364       174
           1     0.9209    0.9368    0.9288       174
           2     0.9551    0.9392    0.9471       181
           3     0.9043    0.9770    0.9392       174
           4     0.9876    0.9191    0.9521       173

    accuracy                         0.9406       876
   macro avg     0.9419    0.9406    0.9407       876
weighted avg     0.9420    0.9406    0.9408       876




This code is solely for educational purposes. The code cannot be used for investments or businesses in practice. Toshi Stats Co.,Ltd. and I do not accept any responsibility or liability for loss or damage occasioned to any person or property through using materials, instructions, methods, algorithm or ideas contained herein, or acting or refraining from acting as a result of such use. Toshi Stats Co.,Ltd. and I expressly disclaim all implied warranties, including merchantability or fitness for any particular purpose. There will be no duty on Toshi Stats Co.,Ltd. and me to correct any errors or defects in the codes and the software

Copyright © 2022 ToshiStats Co.,Ltd. All right reserved